# Morphologic Transformations Visualizations 🔲

This notebook contains algorithms for visualizing some basic morphologic transformations such as erosion, dilation, opening, and closing. 

It serves the purpose of offering a better understanding in the process of exploring Image Processing. 📚

In [42]:
import cv2 as cv
import numpy as np
import os
import imageio # for creating gifs

# Dilation and Erosion

## Out-of-box behavior

In [39]:
input_img = cv.imread("./img/mon1thr1_bw.bmp", cv.IMREAD_GRAYSCALE)
cv.imshow("source", input_img)

default_dilated = cv.dilate(input_img, None, iterations = 1)

default_eroded = cv.erode(input_img, None, iterations = 1)

cv.imshow("default dilation", default_dilated)
cv.imshow("default eroded", default_eroded)

cv.waitKey(0)
cv.destroyWindow("source")
cv.destroyWindow("default dilation")
cv.destroyWindow("default eroded")

*Remark:* The behavior is the opposite way w.r.t the expectation.

## Custom Implementation and Visualization

Requirements:
- the input image is a very small one containing one or more objects (black pixels) over a white background
    - using a large image may lead to memory allocation errors

### Helper functions
The dilation and erosion routines need some additional functions for creating images with the intermmediary steps. These functions are featured below.

In [45]:
def getOffsets(structuring_elem):
    """
    Extract the coordinates of the structuring element cells that contain a 1.
    Return them as a list of tuples
    """
    elem_matrix, origin = structuring_elem
    x_origin, y_origin = origin

    (x_coord, y_coord) = np.where(elem_matrix == 1)    
    x_coord = x_coord - x_origin    
    y_coord = y_coord - y_origin
    
    offsets = list(zip(x_coord, y_coord))
    return offsets
    
def inImage(curr_i, curr_j, rows, cols):
    """
    Check if a pixel at the given coordinates is in the image
    """
    if curr_i > 0 and curr_i < rows:
        if curr_j > 0 and curr_j < cols:
            return True
    return False

def zoom_in(src):
    """
    Return a zoomed in version of the image. The percentage inverse proportional to the area.
    Use nearest neighbor interpolation to preserve sharpness.
    """
    height, width, _ = src.shape
    area = height * width
    percentage = 1290 * 3000 / area # experimentally adjusted values
    new_height = int(height * percentage / 100) 
    new_width = int(width * percentage / 100)
    dst = cv.resize(src, (new_width, new_height), interpolation = cv.INTER_NEAREST)
    return dst

def overlapStructElem(src, acc, src_after, i, j, structuring_elem):
    """
    Create an image that overlaps the structuring element and the image that is being transformed.
    Also adds snapshots of the source image before and after the dilation/erosion overlapped
    with the structuring element.
    """
    offsets = getOffsets(structuring_elem)
    rows, cols = acc.shape
    to_return= np.full((rows, cols, 3), 255, np.uint8) 

    # color object pixels black
    to_return[:, :, 0] = acc
    to_return[:, :, 1] = acc
    to_return[:, :, 2] = acc

    for offset_i, offset_j in offsets: # navigate through the structuring element pixels of value 1
        index_i = i + offset_i
        index_j = j + offset_j
        if (to_return[index_i, index_j, 1] == 0): # intersection between struct elem and object (dark green)
            to_return[index_i, index_j, 1] = 99
            to_return[index_i, index_j, 2] = 18
        else: # structuring element outside object (green)
            to_return[index_i, index_j, 0] = 0
            to_return[index_i, index_j, 2] = 0

    # highlight object center (yellow)
    to_return[i, j, 1] = 255
    to_return[i, j, 2] = 255

    
    """
    Add the snapshots containing source ++ structuring element before and after dilation
    """
    
    elem_matrix, origin = structuring_elem
    x_origin, y_origin = origin
    struct_rows, struct_cols = elem_matrix.shape
    
    padding = 2
    window_start_i = i - x_origin - padding
    window_end_i = i + (struct_rows - x_origin) + padding
    window_start_j = j - y_origin - padding
    window_end_j = j + (struct_cols - y_origin) + padding

    # BEFORE 
    before = np.full((struct_rows + 2 * padding, struct_cols + 2 * padding, 3), 255, np.uint8)
    src_crop = src[window_start_i : window_end_i, window_start_j : window_end_j] # extract region of interest from src

    # Add source object pixels
    before[:, :, 0] = src_crop
    before[:, :, 1] = src_crop
    before[:, :, 2] = src_crop

    # Add structuring element
    for offset_i, offset_j in offsets:
        index_i = x_origin + offset_i + padding
        index_j = y_origin + offset_j + padding
        if(before[index_i, index_j, 1] == 0): # intersection between struct elem and object (dark green)
            before[index_i, index_j, 1] = 99
            before[index_i, index_j, 2] = 18
        else: # structuring element outside object (green)
            before[index_i, index_j, 0] = 0
            before[index_i, index_j, 2] = 0

    # Highlight center if it has the value 1 in the structuring element (yellow)
    if((x_origin, y_origin) in offsets):
        before[x_origin + padding, y_origin + padding, 1] = 255
        before[x_origin + padding, y_origin + padding, 2] = 255

    # Overlap the output and the BEFORE snapshot in the top left corner
    to_return[0 : struct_rows + 2 * padding, 0: struct_cols + 2 * padding, : ] = before
    
    # AFTER
    after = np.full((struct_rows + 2 * padding, struct_cols + 2 * padding, 3), 255, np.uint8)
    src_after_crop = src_after[window_start_i : window_end_i, window_start_j : window_end_j]

    after[:, :, 0] = src_after_crop  
    after[:, :, 1] = src_after_crop    
    after[:, :, 2] = src_after_crop

    # Add structuring element (with offset of 2)
    for offset_i, offset_j in offsets:
        index_i = x_origin + offset_i + 2
        index_j = y_origin + offset_j + 2
        if(after[index_i, index_j, 1] == 0): # intersection between struct elem and object (dark green)
            after[index_i, index_j, 1] = 99
            after[index_i, index_j, 2] = 18
        else: # structuring element outside object (green)
            after[index_i, index_j, 0] = 0
            after[index_i, index_j, 2] = 0

    # Overlap the output and the AFTER snapshot after the before snapshot
    to_return[0 : struct_rows + 2 * padding , struct_cols + 2 * padding + 1 : (struct_cols + 2 * padding + 1 + struct_cols + 2 * padding), :] = after

    # swap the third and first cols (imageio works with RGB not GRB)
    to_return[:, :, [0, 2]] = to_return[:, :, [2, 0]]
    return to_return

def gifFromImages(src_frames, out_path, duration):
    """
    Create a gif from the list of frames src_frames
    """
    imageio.mimsave(os.path.join(out_path), src_frames, duration=duration)
    return


### Implementation of dilation and erosion

In [33]:

def customDilationWithVisualization(src, structuring_elem, iterations):
    """
    Performs a dilation on the source image. It builds a list of images representing
    intermmediate steps through the algorithm
    """
    rows, cols = src.shape
    structuring_elem_offsets = getOffsets(structuring_elem)
    ret_images = []
    
    for _ in range(iterations):
        acc = np.copy(src) # accumulator
        
        for i in range(rows):
            for j in range(cols):
                if src[i, j] == 0:
                    last_i, last_j = i,j # save the last intersection between struc. elem. and object                    
                    
                    acc_cpy = np.copy(acc) # accumulator before the dilation
                    src_post = np.copy(src) # source after the dilation

                    # mark the center as background
                    acc[i, j] = 255 
                    src_post[i, j] = 255
                    
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            acc[i + offset_i, j + offset_j] = 0
                            src_post[i + offset_i, j + offset_j] = 0
                    overlapped = overlapStructElem(src, acc_cpy, src_post, i, j, structuring_elem)
                    ret_images.append(zoom_in(overlapped)) # add to the list of frames    

        overlapped = overlapStructElem(src, acc, src_post, last_i, last_j, structuring_elem)
        ret_images.append(zoom_in(overlapped))
        src = np.copy(acc)

    return ret_images

def customErosionWithVisualization(src, structuring_elem, iterations):
    rows, cols = src.shape
    acc = np.copy(src)
    structuring_elem_offsets = getOffsets(structuring_elem)
    ret_images = []

    for _ in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j]
                if curr_pixel == 0:
                    last_i, last_j = i, j

                    acc_cpy = np.copy(acc)
                    src_after = np.copy(src)

                    all_neigbors_are_object = True
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            if src[i + offset_i, j + offset_j] == 255:
                                all_neigbors_are_object = False
                                break

                    if all_neigbors_are_object:
                        acc[i, j] = 0 # color the origin as object
                        src_after[i, j] = 0
                    else:
                        acc[i, j] = 255
                        src_after[i, j] = 255
                    
                    ovelapped = overlapStructElem(src, acc_cpy, src_after, i, j, structuring_elem)
                    ret_images.append(zoom_in(ovelapped))
        
        ovelapped = overlapStructElem(src, acc, src_after, last_i, last_j, structuring_elem)
        ret_images.append(zoom_in(ovelapped))

        src = np.copy(acc)

    return ret_images

### Generating the desired gifs

In [47]:
def main(isDilation, input_path, structuring_element, iterations, output_path, frame_duration) :
    input_img = cv.imread(input_path, cv.IMREAD_GRAYSCALE)

    if isDilation:    
        frames = customDilationWithVisualization(input_img, structuring_element, iterations)
    else:   
        frames = customErosionWithVisualization(input_img, structuring_element, iterations)
    
    gifFromImages(frames, output_path, frame_duration)

#### Dilation Gif

In [50]:
if __name__ == '__main__':
    struct_element = (np.array([[1,1,1],[1,1,1],[1,1,1]]), (1, 1))
    main(True, "./img/square.bmp", struct_element, 1, "./outDilation/dilation_gif.gif", 300)

#### Erosion Gif

In [52]:
if __name__ == '__main__':
    struct_element = (np.array([[1,1,1],[1,1,1],[1,1,1]]), (1, 1))
    main(False, "./img/infinity.bmp", struct_element, 1, "./outErosion/erosion_gif.gif", 300)

---

### Next Steps
- add region filling